In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets

In [3]:
train_data = datasets.load_boston()

In [6]:
type(train_data)

sklearn.datasets.base.Bunch

In [11]:
features = train_data.feature_names
print features

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [20]:
# obtain features and target
data = train_data.data
target = train_data.target
